In [1]:
# slide download

In [42]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
ds = load_dataset("rotten_tomatoes")

Found cached dataset rotten_tomatoes (/home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
from transformers import AutoModel
model_name = "distilbert-base-uncased"
model = AutoModel.from_pretrained(model_name)
model

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [56]:
batch = ds_encoded["train"][:2]
input_ids = torch.tensor(batch["input_ids"])
attention_mask = torch.tensor(batch["attention_mask"])


In [62]:
with torch.no_grad():
    output = model(input_ids, attention_mask)
    lhs = output.last_hidden_state.cpu().numpy()
    
lhs.shape

(2, 78, 768)

In [44]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

ds_encoded = ds.map(tokenize, batched=True, batch_size=None)

model(
    torch.tensor(ds_encoded["train"][0]["input_ids"]).reshape(1, -1),
    torch.tensor(ds_encoded["train"][0]["attention_mask"]).reshape(1, -1),
)

Loading cached processed dataset at /home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-88982f87eaae9cb2.arrow
Loading cached processed dataset at /home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-6af834e10e455eab.arrow
Loading cached processed dataset at /home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-a5726ca55adbf819.arrow


BaseModelOutput(last_hidden_state=tensor([[[-0.0332, -0.0168,  0.0194,  ...,  0.0476,  0.5834,  0.3036],
         [-0.0235, -0.0555, -0.3638,  ...,  0.1877,  0.5781, -0.1577],
         [-0.0516, -0.1014, -0.1511,  ...,  0.1503,  0.2649, -0.1575],
         ...,
         [ 0.1135, -0.0297,  0.3137,  ...,  0.0454,  0.2319, -0.1062],
         [ 0.4841, -0.1742,  0.3653,  ...,  0.1134, -0.0638, -0.2683],
         [ 0.3589, -0.0675,  0.4041,  ...,  0.0591,  0.2657, -0.2713]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [17]:
def f(row):
    row["new_variable"] = row["text"].split(" ")[0]
    return row

new_ds = ds.map(f)
new_ds.column_names

Loading cached processed dataset at /home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-11bd3644f874830b.arrow
Loading cached processed dataset at /home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-4fe48785846da2d6.arrow
Loading cached processed dataset at /home/janos/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-3c27a681b77b3ca3.arrow


{'train': ['text', 'label', 'new_variable'],
 'validation': ['text', 'label', 'new_variable'],
 'test': ['text', 'label', 'new_variable']}

In [20]:
ds.set_format(type="pandas")
ds.map(f)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

AttributeError: 'Series' object has no attribute 'split'

In [22]:
def f(row):
    row["first_word"] = row["text"].split(" ")[0]
    return row

f(ds["train"])[0]

AttributeError: 'Series' object has no attribute 'split'

In [9]:
mapped_ds["train"].column_names

['text', 'label', 'first_word']

In [5]:
ds.set_format("pandas")
ds.map?

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [105]:
ds.set_format(None)
ds.map(f)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'first_word'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'first_word'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'first_word'],
        num_rows: 1066
    })
})